# 2.7 Upper-Confidence_Bound Action Selection

In [ ]:
import numpy as np
import os
from tqdm import tqdm_notebook

np.random.seed(77)

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15
plt.rcParams["figure.figsize"] = (11,4)

plt.rcParams['axes.unicode_minus'] = False

PROJECT_ROOT_DIR = "."
CHAPTER_ID = '10_armed_testbed'

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [ ]:
def bandit(action):
    return np.random.normal(q_rewards[action],1)

def greedy_action_selection(t, q, epsilon = 0):

    rand = np.random.rand(1)[0]

    if t <= 0 or epsilon >= rand:            
        return np.random.choice(NUM_OF_ACTION,1)[0]
    else:
        return argmax(q)

def UCB_action_selection(t, q, c, n):
    
    tmp_q = q.copy()
    
    if t <= 0:            
        return np.random.choice(NUM_OF_ACTION,1)[0]
    else:
        for a in range(NUM_OF_ACTION):
            if n[a] == 0:
                tmp_q[a] = np.inf
            else:
                tmp_q[a] += c*np.sqrt(np.log(t)/n[a])
        return argmax(tmp_q)
    
def argmax(array):
    top_index = [0]
    top = array[0]
    
    for i in range(1,len(array)):
        if array[i] > top:
            top_index = [i]
            top = array[i]
        elif array[i] == top:
            top_index.append(i)
        
    np.random.shuffle(top_index)
        
    return top_index[0]

In [ ]:
## simple bandit algorithm with step-size parameter, α = 0.1

STEP = 1000
RUN = 2000
NUM_OF_ACTION = 10

q_rewards = []
for _ in range(NUM_OF_ACTION):
    tmp = np.random.normal(0,1)
    q_rewards.append(tmp)
    
optimal_action = np.argmax(q_rewards)

UCB_OPTIMAL_ACTIONS = [0]*STEP
UCB_AVERAGE_REWARDS = [0]*STEP

e01_OPTIMAL_ACTIONS = [0]*STEP
e01_AVERAGE_REWARDS = [0]*STEP

## 10-armed testbed
for i in tqdm_notebook(range(RUN)):

    # Upper-Confidence_Bound Action Selection with c = 2
    Q = [0]*NUM_OF_ACTION
    N = [0]*NUM_OF_ACTION

    for t in range(STEP):
        A = UCB_action_selection(t, Q, 2, N)
        R = bandit(A)
        N[A] += 1
        Q[A] += (1/N[A])*(R-Q[A])
        
        UCB_AVERAGE_REWARDS[t] += R/RUN
        if A == optimal_action:
            UCB_OPTIMAL_ACTIONS[t] += 1/RUN*100
            
    # greedy ε = 0.1
    Q = [0]*NUM_OF_ACTION
    N = [0]*NUM_OF_ACTION

    for t in range(STEP):
        A = greedy_action_selection(t, Q, 0.1)
        R = bandit(A)
        N[A] +=1
        Q[A] += (1/N[A])*(R-Q[A])
        
        e01_AVERAGE_REWARDS[t] += R/RUN
        if A == optimal_action:
            e01_OPTIMAL_ACTIONS[t] += 1/RUN*100

In [ ]:
plt.plot(np.arange(STEP), UCB_AVERAGE_REWARDS, 'b-', np.arange(STEP), e01_AVERAGE_REWARDS,'k-')
plt.legend(["UCB c=2", "ε-greedy ε=0.1"])
plt.xlabel("Steps")
plt.ylabel("Average reward")
save_fig("UCB_average_reward")
plt.show()